<a href="https://colab.research.google.com/github/ErikaDenisse29/ColabFiles/blob/main/FramesConsultaInventarios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 Representación de Conocimiento Basada en Frames (con herencia,
agregación, demonios y reglas)

In [5]:
class FrameSystem:

    def __init__(self):
        self.frames = {}
        self.tipos = {}
        self.slots_especificacion = {}
        self._calculando = set()
        self.reglas = {}
        self.hechos_inferidos = set()

    def define_tipo(self, nombre, supertipo=None):
        if nombre not in self.frames:
            self.frames[nombre] = {
                'tipo': 'clase',
                'nombre': nombre,
                'supertipo': supertipo,
                'slots': {}
            }
            self.tipos[nombre] = self.frames[nombre]
        return nombre

    def define_slot(self, tipo_nombre, slot_nombre, **especificacion):
        if tipo_nombre not in self.tipos:
            raise ValueError(f"Tipo {tipo_nombre} no existe")

        if tipo_nombre not in self.slots_especificacion:
            self.slots_especificacion[tipo_nombre] = {}

        self.slots_especificacion[tipo_nombre][slot_nombre] = especificacion


        if 'default' in especificacion:
            self.frames[tipo_nombre]['slots'][slot_nombre] = especificacion['default']

    def crear_instancia(self, nombre_instancia, tipo_nombre):
        if tipo_nombre not in self.tipos:
            raise ValueError(f"Tipo {tipo_nombre} no existe")

        if nombre_instancia in self.frames:
            raise ValueError(f"Ya existe un frame con nombre {nombre_instancia}")

        self.frames[nombre_instancia] = {
            'tipo': 'instancia',
            'nombre': nombre_instancia,
            'tipo_base': tipo_nombre,
            'slots': {}
        }

        return nombre_instancia

    def get_slots_herencia(self, nombre_frame):
        if nombre_frame not in self.frames:
            raise ValueError(f"Frame {nombre_frame} no existe")

        frame = self.frames[nombre_frame]
        slots_heredados = {}

        if frame['tipo'] == 'instancia':
            tipo_actual = frame['tipo_base']
        else:
            tipo_actual = nombre_frame


        tipos_recorridos = set()
        while tipo_actual and tipo_actual not in tipos_recorridos:
            tipos_recorridos.add(tipo_actual)

            if tipo_actual in self.slots_especificacion:
                for slot_nombre, espec in self.slots_especificacion[tipo_actual].items():
                    if slot_nombre not in slots_heredados:
                        if 'default' in espec:
                            slots_heredados[slot_nombre] = espec['default']
                        else:
                            slots_heredados[slot_nombre] = None

            if tipo_actual in self.tipos and self.tipos[tipo_actual]['supertipo']:
                tipo_actual = self.tipos[tipo_actual]['supertipo']
            else:
                tipo_actual = None

        return slots_heredados

    def set_slot(self, nombre_frame, slot_nombre, valor):
        if nombre_frame not in self.frames:
            raise ValueError(f"Frame {nombre_frame} no existe")

        frame = self.frames[nombre_frame]

        tipo_base = frame['tipo_base'] if frame['tipo'] == 'instancia' else nombre_frame
        if self._check_allowed(tipo_base, slot_nombre, valor):
            frame['slots'][slot_nombre] = valor
            print(f"Slot '{slot_nombre}' de '{nombre_frame}' establecido a: {valor}")

            self._ejecutar_demonios_onset(nombre_frame, slot_nombre, valor)
        else:
            print(f"Error: Valor '{valor}' no permitido para slot '{slot_nombre}'")

    def _check_allowed(self, tipo_nombre, slot_nombre, valor):
        tipo_actual = tipo_nombre
        tipos_revisados = set()

        while tipo_actual and tipo_actual not in tipos_revisados:
            tipos_revisados.add(tipo_actual)

            if tipo_actual in self.slots_especificacion:
                if slot_nombre in self.slots_especificacion[tipo_actual]:
                    espec = self.slots_especificacion[tipo_actual][slot_nombre]
                    if 'allowed' in espec:
                        return valor in espec['allowed']
                    return True


            if tipo_actual in self.tipos and self.tipos[tipo_actual]['supertipo']:
                tipo_actual = self.tipos[tipo_actual]['supertipo']
            else:
                tipo_actual = None

        return True

    def get_slot(self, nombre_frame, slot_nombre):
        if nombre_frame not in self.frames:
            raise ValueError(f"Frame {nombre_frame} no existe")

        frame = self.frames[nombre_frame]
        if slot_nombre in frame['slots']:
            return frame['slots'][slot_nombre]

        slots_heredados = self.get_slots_herencia(nombre_frame)
        if slot_nombre in slots_heredados:
            return slots_heredados[slot_nombre]

        return self._ejecutar_demonios_ifneeded(nombre_frame, slot_nombre)

    def _ejecutar_demonios_ifneeded(self, nombre_frame, slot_nombre):
        print(f"Demonio if-needed: calculando '{slot_nombre}' para '{nombre_frame}'")

        if slot_nombre in self._calculando:
            print(f"  Detectado ciclo en cálculo de '{slot_nombre}', retornando None")
            return None

        self._calculando.add(slot_nombre)

        try:
            if slot_nombre == 'intencion_interpretada':
                texto = self.get_slot(nombre_frame, 'texto_consulta')
                if texto:
                    texto_lower = texto.lower()
                    if "crítico" in texto_lower or "critico" in texto_lower:
                        valor = "stock_critico"
                    elif "valor" in texto_lower or "total" in texto_lower:
                        valor = "valor_inventario"
                    elif "bajo" in texto_lower:
                        valor = "stock_bajo"
                    else:
                        valor = "consulta_general"

                    print(f"Intención interpretada: '{valor}' (basado en: '{texto}')")
                    self.frames[nombre_frame]['slots'][slot_nombre] = valor
                    return valor


            elif slot_nombre == 'sql_generado':
                intencion = self.get_slot(nombre_frame, 'intencion_interpretada')
                if intencion:
                    if intencion == "stock_critico":
                        sql = "SELECT * FROM productos WHERE stock < stock_minimo;"
                    elif intencion == "valor_inventario":
                        sql = "SELECT SUM(precio * stock) FROM productos;"
                    elif intencion == "stock_bajo":
                        sql = "SELECT * FROM productos WHERE stock < 10;"
                    else:
                        sql = "SELECT * FROM productos LIMIT 100;"

                    print(f"  → SQL generado: '{sql}'")
                    self.frames[nombre_frame]['slots'][slot_nombre] = sql
                    return sql

            print(f"No se pudo calcular '{slot_nombre}'")
            return None

        finally:
            self._calculando.remove(slot_nombre)

    def _ejecutar_demonios_onset(self, nombre_frame, slot_modificado, valor):

        if slot_modificado == 'estado':
            import time
            timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
            self.frames[nombre_frame]['slots']['timestamp_ultimo_cambio'] = timestamp
            print(f"Demonio on-set: 'estado' cambió → actualizando 'timestamp_ultimo_cambio' a {timestamp}")

        elif slot_modificado == 'prioridad':
            if valor == 'alta' or valor == 'critica':
                self.frames[nombre_frame]['slots']['nivel_urgencia'] = 'alto'
                print(f"Demonio on-set: 'prioridad' cambió → actualizando 'nivel_urgencia' a alto")
            else:
                self.frames[nombre_frame]['slots']['nivel_urgencia'] = 'normal'
                print(f"Demonio on-set: 'prioridad' cambió → actualizando 'nivel_urgencia' a normal")

    def agregar_relacion_parte(self, todo, parte, tipo_relacion="tiene_parte"):
        if todo not in self.frames or parte not in self.frames:
            raise ValueError("Uno de los frames no existe")

        if tipo_relacion == "tiene_parte":
            if 'tiene_parte' not in self.frames[todo]['slots']:
                self.frames[todo]['slots']['tiene_parte'] = []
            if parte not in self.frames[todo]['slots']['tiene_parte']:
                self.frames[todo]['slots']['tiene_parte'].append(parte)

            if 'parte_de' not in self.frames[parte]['slots']:
                self.frames[parte]['slots']['parte_de'] = []
            if todo not in self.frames[parte]['slots']['parte_de']:
                self.frames[parte]['slots']['parte_de'].append(todo)

            print(f"Relación '{tipo_relacion}' establecida: {todo} → {parte}")

    def is_a(self, subtipo, supertipo):
        if subtipo not in self.tipos:
            return False

        actual = self.tipos[subtipo]
        while actual:
            if actual['supertipo'] == supertipo:
                return True
            if actual['supertipo'] and actual['supertipo'] in self.tipos:
                actual = self.tipos[actual['supertipo']]
            else:
                break
        return False

    def agregar_regla(self, nombre_regla, condiciones, acciones):
        self.reglas[nombre_regla] = {
            'condiciones': condiciones,
            'acciones': acciones,
            'disparada': False
        }
        print(f"Regla '{nombre_regla}' agregada")

    def inferir(self, max_iteraciones=3):
        iteracion = 1
        cambios_globales = False

        while iteracion <= max_iteraciones:
            print(f"\nIteración {iteracion}:")
            cambios_en_iteracion = False

            for nombre_regla, regla in self.reglas.items():
                if regla.get('disparada', False):
                    continue

                print(f"  Evaluando regla: {nombre_regla}")


                condiciones_cumplidas = True
                for cond in regla['condiciones']:
                    if not self._evaluar_condicion(cond):
                        condiciones_cumplidas = False
                        print(f"    Condición no cumplida: {cond}")
                        break
                    else:
                        print(f"    Condición cumplida: {cond}")


                if condiciones_cumplidas:
                    print(f"    REGLA DISPARADA: {nombre_regla}")
                    for accion in regla['acciones']:
                        resultado = self._ejecutar_accion(accion)
                        if resultado:
                            cambios_en_iteracion = True


                    self.reglas[nombre_regla]['disparada'] = True

            if not cambios_en_iteracion:
                print("\n  No más cambios. Inferencia terminada.")
                break

            cambios_globales = True
            iteracion += 1

        if iteracion > max_iteraciones:
            print(f"\n  Límite de {max_iteraciones} iteraciones alcanzado.")

        # Resetear banderas para futuras inferencias
        for nombre_regla in self.reglas:
            self.reglas[nombre_regla]['disparada'] = False

    def _evaluar_condicion(self, condicion):
        try:

            if len(condicion) == 4:
                frame, slot, op, valor = condicion
                valor_actual = self.get_slot(frame, slot)


                if valor is None:
                    if op == '==':
                        return valor_actual is None
                    elif op == '!=':
                        return valor_actual is not None

                if op == '==':
                    return valor_actual == valor
                elif op == '>':
                    return valor_actual > valor
                elif op == '<':
                    return valor_actual < valor
                elif op == '!=':
                    return valor_actual != valor
            return False
        except Exception as e:
            print(f"    Error evaluando condición: {e}")
            return False

    def _ejecutar_accion(self, accion):

        try:
            if accion[0] == 'set' and len(accion) == 4:
                _, frame, slot, valor = accion
                self.set_slot(frame, slot, valor)
                print(f"    Acción ejecutada: set({frame}, {slot}, {valor})")
                return True
        except Exception as e:
            print(f"     Error en acción: {e}")
        return False

    def mostrar_info(self, nombre_frame):

        if nombre_frame not in self.frames:
            print(f"Frame '{nombre_frame}' no existe")
            return

        frame = self.frames[nombre_frame]
        print(f"FRAME: {nombre_frame}")
        print(f"Tipo: {frame['tipo']}")
        if frame['tipo'] == 'instancia':
            print(f"Tipo base: {frame['tipo_base']}")


        print("\nSlots locales:")
        if frame['slots']:
            for slot, valor in sorted(frame['slots'].items()):
                print(f"  • {slot}: {valor}")
        else:
            print("  (vacío)")

        if frame['tipo'] == 'instancia':
            print("\nSlots heredados:")
            heredados = self.get_slots_herencia(nombre_frame)
            if heredados:
                for slot, valor in sorted(heredados.items()):
                    if slot not in frame['slots']:
                        print(f"  • {slot}: {valor} (heredado)")
            else:
                print("  (ninguno)")



def implementar_dominio():

    print(" IMPLEMENTACIÓN DE FRAMES - SISTEMA DE CONSULTAS DE INVENTARIO")

    sistema = FrameSystem()

    print("\n1. DEFINICIÓN DE TIPOS (Jerarquía IS-A)")
    print("-" * 50)


    sistema.define_tipo('Usuario')
    sistema.define_tipo('Consulta')
    sistema.define_tipo('Reporte')
    sistema.define_tipo('CanalEntrega')

    sistema.define_tipo('UsuarioAvanzado', 'Usuario')
    sistema.define_tipo('ConsultaEspecializada', 'Consulta')
    sistema.define_tipo('ReporteEjecutivo', 'Reporte')

    sistema.define_tipo('Gerente', 'UsuarioAvanzado')
    sistema.define_tipo('Analista', 'UsuarioAvanzado')
    sistema.define_tipo('Ejecutivo', 'UsuarioAvanzado')
    sistema.define_tipo('ConsultaStock', 'ConsultaEspecializada')
    sistema.define_tipo('ConsultaValor', 'ConsultaEspecializada')
    sistema.define_tipo('ReporteInventario', 'ReporteEjecutivo')
    sistema.define_tipo('ReporteVentas', 'ReporteEjecutivo')

    print("Tipos definidos (14 tipos):")
    print("  • Nivel 1: Usuario, Consulta, Reporte, CanalEntrega")
    print("  • Nivel 2: UsuarioAvanzado, ConsultaEspecializada, ReporteEjecutivo")
    print("  • Nivel 3: Gerente, Analista, Ejecutivo, ConsultaStock, ConsultaValor, ReporteInventario, ReporteVentas")


    print("\n2. DEFINICIÓN DE SLOTS")



    sistema.define_slot('Usuario', 'nombre', default="Usuario Anónimo")
    sistema.define_slot('Usuario', 'activo', default=True)
    sistema.define_slot('Consulta', 'estado', default="pendiente", allowed=["pendiente", "procesando", "completada", "error"])

    sistema.define_slot('CanalEntrega', 'tipo', allowed=["ejecutivo", "detalle", "mayoreo"])
    sistema.define_slot('Reporte', 'formato', allowed=["PDF", "Excel", "HTML", "CSV"])

    sistema.define_slot('Consulta', 'texto_consulta')
    sistema.define_slot('ConsultaEspecializada', 'prioridad', default="media", allowed=["baja", "media", "alta", "critica"])
    sistema.define_slot('Gerente', 'nivel_acceso', default="alto")
    sistema.define_slot('Analista', 'especialidad')
    sistema.define_slot('Reporte', 'fecha_generacion')

    print("Slots definidos (10 slots):")
    print("  • 3 slots con default: nombre, activo, estado, prioridad")
    print("  • 2 slots con allowed: tipo (CanalEntrega), formato (Reporte), estado (Consulta), prioridad (ConsultaEspecializada)")


    print("\n3. CREACIÓN DE INSTANCIAS")



    sistema.crear_instancia('Carlos_Gerente', 'Gerente')
    sistema.set_slot('Carlos_Gerente', 'nombre', 'Carlos Rodríguez')
    sistema.set_slot('Carlos_Gerente', 'nivel_acceso', 'máximo')

    sistema.crear_instancia('Ana_Analista', 'Analista')
    sistema.set_slot('Ana_Analista', 'nombre', 'Ana Martínez')
    sistema.set_slot('Ana_Analista', 'especialidad', 'inventario')

    sistema.crear_instancia('Pedro_Ejecutivo', 'Ejecutivo')
    sistema.set_slot('Pedro_Ejecutivo', 'nombre', 'Pedro Sánchez')

    sistema.crear_instancia('Consulta_001', 'ConsultaStock')
    sistema.set_slot('Consulta_001', 'texto_consulta', 'Mostrar productos con stock crítico')
    sistema.set_slot('Consulta_001', 'estado', 'pendiente')
    sistema.set_slot('Consulta_001', 'prioridad', 'alta')

    sistema.crear_instancia('Consulta_002', 'ConsultaValor')
    sistema.set_slot('Consulta_002', 'texto_consulta', 'Calcular valor total del inventario')
    sistema.set_slot('Consulta_002', 'estado', 'pendiente')


    import time
    sistema.crear_instancia('Reporte_Inv_001', 'ReporteInventario')
    sistema.set_slot('Reporte_Inv_001', 'formato', 'PDF')
    sistema.set_slot('Reporte_Inv_001', 'fecha_generacion', time.strftime("%Y-%m-%d"))

    print("Instancias creadas (6):")
    print("  • Carlos_Gerente (Gerente)")
    print("  • Ana_Analista (Analista)")
    print("  • Pedro_Ejecutivo (Ejecutivo)")
    print("  • Consulta_001 (ConsultaStock)")
    print("  • Consulta_002 (ConsultaValor)")
    print("  • Reporte_Inv_001 (ReporteInventario)")


    print("\n4. RELACIONES PARTE-TODO (Agregación)")


    sistema.agregar_relacion_parte('Carlos_Gerente', 'Consulta_001')
    sistema.agregar_relacion_parte('Carlos_Gerente', 'Consulta_002')
    sistema.agregar_relacion_parte('Ana_Analista', 'Consulta_001')

    sistema.agregar_relacion_parte('Consulta_001', 'Reporte_Inv_001')

    print("\nEstructura de agregación de 2 niveles:")
    print("  Nivel 1: Carlos_Gerente tiene_parte [Consulta_001, Consulta_002]")
    print("  Nivel 2: Consulta_001 tiene_parte Reporte_Inv_001")
    print("  También: Ana_Analista tiene_parte Consulta_001")


    print("\n5. REGLAS DE INFERENCIA")

    sistema.agregar_regla(
        'detectar_consulta_critica',
        [('Consulta_001', 'prioridad', '==', 'alta')],
        [('set', 'Consulta_001', 'etiqueta_riesgo', 'critica')]
    )

    sistema.agregar_regla(
        'generar_alerta_stock',
        [('Consulta_001', 'etiqueta_riesgo', '==', 'critica')],
        [('set', 'Carlos_Gerente', 'alerta_activa', True),
         ('set', 'Carlos_Gerente', 'accion_recomendada', 'revisar_stock_inmediato')]
    )

    print("Reglas definidas:")
    print("  • Regla 1 (etiqueta): detectar_consulta_critica")
    print("  • Regla 2 (acción): generar_alerta_stock")

    return sistema



def ejecutar_pruebas(sistema):
    print("\nPRUEBA 1: HERENCIA (IS-A)")

    print(f"is_a('Gerente', 'Usuario'): {sistema.is_a('Gerente', 'Usuario')}")
    print(f"is_a('Gerente', 'UsuarioAvanzado'): {sistema.is_a('Gerente', 'UsuarioAvanzado')}")
    print(f"is_a('Analista', 'Gerente'): {sistema.is_a('Analista', 'Gerente')} (debería ser False)")

    print("\nSlot heredado en instancia:")
    nombre = sistema.get_slot('Ana_Analista', 'nombre')
    print(f"  get_slot('Ana_Analista', 'nombre') = '{nombre}' (heredado de Usuario)")

    activo = sistema.get_slot('Pedro_Ejecutivo', 'activo')
    print(f"  get_slot('Pedro_Ejecutivo', 'activo') = {activo} (heredado de Usuario)")

    print("\nPRUEBA 2: AGREGACIÓN (PARTE-TODO)")

    print("Relaciones tiene_parte:")
    partes_carlos = sistema.get_slot('Carlos_Gerente', 'tiene_parte')
    print(f"  Carlos_Gerente tiene_parte: {partes_carlos}")

    partes_consulta = sistema.get_slot('Consulta_001', 'tiene_parte')
    print(f"  Consulta_001 tiene_parte: {partes_consulta}")

    print("\nRelaciones parte_de:")
    parte_de_reporte = sistema.get_slot('Reporte_Inv_001', 'parte_de')
    print(f"  Reporte_Inv_001 parte_de: {parte_de_reporte}")


    print("\nPRUEBA 3: DEMONIO IF-NEEDED")


    print("Consultando slot no existente 'intencion_interpretada' en Consulta_001:")
    intencion = sistema.get_slot('Consulta_001', 'intencion_interpretada')
    print(f"  Resultado: '{intencion}'")

    print("\nConsultando slot no existente 'sql_generado' en Consulta_001:")
    sql = sistema.get_slot('Consulta_001', 'sql_generado')
    print(f"  Resultado: '{sql}'")

    print("\nSegunda llamada a 'sql_generado' (debe usar valor guardado):")
    sql2 = sistema.get_slot('Consulta_001', 'sql_generado')
    print(f"  Resultado: '{sql2}' (sin demonio - valor en cache)")


    print("\nPRUEBA 4: DEMONIO ON-SET")

    print("Cambiando estado de consulta (debe activar demonio on-set #1):")
    sistema.set_slot('Consulta_001', 'estado', 'completada')

    timestamp = sistema.get_slot('Consulta_001', 'timestamp_ultimo_cambio')
    print(f"  Timestamp actualizado: {timestamp}")

    print("\nCambiando prioridad de consulta (debe activar demonio on-set #2):")
    sistema.set_slot('Consulta_001', 'prioridad', 'critica')

    urgencia = sistema.get_slot('Consulta_001', 'nivel_urgencia')
    print(f"  Nivel de urgencia actualizado: {urgencia}")


    print("\nPRUEBA 5: REGLAS DE INFERENCIA")
    sistema.inferir()

    print("\nResultados de la inferencia:")
    etiqueta = sistema.get_slot('Consulta_001', 'etiqueta_riesgo')
    print(f"  Consulta_001 - etiqueta_riesgo: {etiqueta}")

    alerta = sistema.get_slot('Carlos_Gerente', 'alerta_activa')
    accion = sistema.get_slot('Carlos_Gerente', 'accion_recomendada')
    print(f"  Carlos_Gerente - alerta_activa: {alerta}")
    print(f"  Carlos_Gerente - accion_recomendada: {accion}")

    print("\nPRUEBA 6: INFORMACIÓN COMPLETA DE FRAMES")

    sistema.mostrar_info('Carlos_Gerente')
    sistema.mostrar_info('Consulta_001')
    sistema.mostrar_info('Reporte_Inv_001')


if __name__ == "__main__":

    sistema = implementar_dominio()
    ejecutar_pruebas(sistema)



 IMPLEMENTACIÓN DE FRAMES - SISTEMA DE CONSULTAS DE INVENTARIO

1. DEFINICIÓN DE TIPOS (Jerarquía IS-A)
--------------------------------------------------
Tipos definidos (14 tipos):
  • Nivel 1: Usuario, Consulta, Reporte, CanalEntrega
  • Nivel 2: UsuarioAvanzado, ConsultaEspecializada, ReporteEjecutivo
  • Nivel 3: Gerente, Analista, Ejecutivo, ConsultaStock, ConsultaValor, ReporteInventario, ReporteVentas

2. DEFINICIÓN DE SLOTS
Slots definidos (10 slots):
  • 3 slots con default: nombre, activo, estado, prioridad
  • 2 slots con allowed: tipo (CanalEntrega), formato (Reporte), estado (Consulta), prioridad (ConsultaEspecializada)

3. CREACIÓN DE INSTANCIAS
Slot 'nombre' de 'Carlos_Gerente' establecido a: Carlos Rodríguez
Slot 'nivel_acceso' de 'Carlos_Gerente' establecido a: máximo
Slot 'nombre' de 'Ana_Analista' establecido a: Ana Martínez
Slot 'especialidad' de 'Ana_Analista' establecido a: inventario
Slot 'nombre' de 'Pedro_Ejecutivo' establecido a: Pedro Sánchez
Slot 'texto_con